<a href="https://colab.research.google.com/github/maxigaarp/Gestion-De-Datos-en-R/blob/main/Proyecto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Proyecto Grupal 

## Entrega: Domingo 29 de Agosto 2021

Este trabajo consistirá en armar una base de datos en SQL y un informe descriptivo de los datos y la investigación. El objetivo del proyecto es predecir asistencia a clases en base a los datos meteorologicos de precipitación.

La base de datos debe tener la informacion de 3 diferentes fuentes de información:

 * Asistencia Declarada Mensual ([MINEDUC](http://datos.mineduc.cl/dashboards/19844/asistencia-declarada-mensual-por-ano/)) años: (2015-2019)
  Acá se incluyen los porcentajes de asistencia y la cantidad de dias trabajados de todos los alumnos en los distintos colegios de Chile 
 * Reportes de precipitacion Mensual ([CR2](https://www.cr2.cl/camels-cl/)): Datos historicos de precipitacion las distintas estaciones meteorologicas en Chile con localizacion geografica.
 * Localizacion Colegios (Propia/Google Maps):
 Datos de la ubicacion geografica de los colegios en Chile.

Para esto generar esta base de datos se espera que se incluyan a lo menos 3 diferentes tablas:
 * alumnos: Informacion relevante de los alumnos. A lo menos incluir 3 atributos.
 * colegios: Informacion relevante de los colegios. A lo menos incluir 3 atributos, ademas:  nombre (o ID) de la estacion meteorologica más cercana y la distancia a esta, asumiendo latitud y longitud como coordenadas cartesianas.
 * estudia_en: Informacion relevante del vinculo entre el colegio, mes y alumnos. En la tabla se debe incorporar la informacion mensual para cada alumno en terminos de: asistencia, precipitacion. En este caso la precipitacion debe ser una interpolacion en base a los datos de las precipitaciones de las estaciones meteorologicas y la geolocalizacion de los colegios.

Cabe destacar que las bases de datos no se encuentran libres de errores por lo que para consolidar la informacion hay que solucionar problemas de consistencia, completitud, validez. Entre otros se busca que:

 * Que los datos para cada uno atributos se encuentren en el formato adecuado y unico. Para lograr esto revisar los atributos más importantes: asistencia y precipitacion.
 * Una unica forma de expresar valores nulos (unificar distintos tipos de descripcion de valores nulos, ejemplo: " ", NA, 0, -9999, etc). 
 * Seleccionar una cantidad tolerable (mayor que 2 al menos) de valores nulos para las series temporales (los registros de las estaciones meteorologicas). Interpolar los valores nulos en las series.

Cabe destacar que las tablas deben unificarse en SQL cargando las tablas mediante RSQL. Luego de esto se espera que se seleccione y utilice un algoritmo adecuado para generar una predicción de asistencia a clases en base a la precipitacion mensual, puede ser alguno de los siguientes: k-means, regresion lineal, regresion logistica.

La entrega del proyecto debe tener dos partes: 
* El codigo donde se genera la base de datos y el desarrollo del algoritmo de prediccion. Puede ser Colab o script de R, debe tener comentarios solo con el objetivo de guiar al ayudante en la correccion.
* Un documento de no mas de 3 paginas (sin incluir gráficos) con la documentacion de los datos (diagrama ER, tipo de datos, descripcion, dominio, etc. Pueden guiarse por el esquema de registro en las bases de datos del MINEDUC.) y además un resumen de lo desarrollado en el proyecto (como se generaron las variables, como se procesaron y que resultados se obtuvieron)

Para una mejor calidad en el informe usar graficos y tablas que pueden incluir en anexos.



Datos disponibles en: https://drive.google.com/drive/folders/17Or8k6rhYvkaeEn_pD10za8A9jluYfUF?usp=sharing

#Descargar datos

##Asistencia

In [2]:
system("gdown --id 1-q1ydcu6afA3LQ9uxlh9J9B9kvEJvrRs")

In [3]:
unzip("/content/DatosProyecto.zip")

In [5]:
library(stringr)

In [ ]:
lista <- list.files("/content/content/drive/MyDrive/Gestion de Datos/Datos Proyecto/Datos asistencia")
directorio <- "/content/content/drive/MyDrive/Gestion de Datos/Datos Proyecto/Datos asistencia/"

for (name in lista){
  if (str_detect(name, ".rar")){
    foldername<-substr(name, 1, nchar(name)-4)
    system(paste("mkdir 'Asistencia/",foldername,"'",sep=""))
    system(paste("unrar x '",directorio,name,"' 'Asistencia/",substr(name, 1, nchar(name)-4),"/'", sep=""),intern = T)
  }
  else {
    unzip(paste(directorio,name,sep=""), exdir="Asistencia")
  }
}


In [21]:
lista_de_csvs <- list.files("Asistencia",pattern = ".(CSV|csv)$", recursive = TRUE))


[1] 50

## Precipitacion

In [22]:
unzip("/content/content/drive/MyDrive/Gestion de Datos/Datos Proyecto/cr2_prAmon_2019.zip")

In [ ]:
read.csv("/content/cr2_prAmon_2019/cr2_prAmon_2019.txt",na = "-9999", header =F)

## Geolocalización




In [27]:
read.csv("/content/content/drive/MyDrive/Gestion de Datos/Datos Proyecto/colesgeo.csv", row.names=1)

,RBD,NOM_RBD,NOM_COM_RBD,query,lon,lat
,<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,1,LICEO POLITECNICO ARICA,ARICA,"LICEO POLITECNICO ARICA , ARICA",-70.29469,-18.48719
2,3,ESC. PEDRO VICENTE GUTIERREZ TORRES,ARICA,"ESC. PEDRO VICENTE GUTIERREZ TORRES , ARICA",-70.30278,-18.48696
3,4,LICEO OCTAVIO PALMA PEREZ,ARICA,"LICEO OCTAVIO PALMA PEREZ , ARICA",-70.30792,-18.47417
4,5,JOVINA NARANJO FERNANDEZ,ARICA,"JOVINA NARANJO FERNANDEZ , ARICA",-70.31345,-18.47680
5,7,L. POLI. ANTONIO VARAS DE LA BARRA,ARICA,"L. POLI. ANTONIO VARAS DE LA BARRA , ARICA",-70.28795,-18.47595
6,8,COLEGIO INTEGRADO EDUARDO FREI MONTALVA,ARICA,"COLEGIO INTEGRADO EDUARDO FREI MONTALVA , ARICA",-70.31737,-18.48267
7,9,ESCUELA REPUBLICA DE ISRAEL,ARICA,"ESCUELA REPUBLICA DE ISRAEL , ARICA",-70.30278,-18.48696
8,10,ESCUELA REPUBLICA DE FRANCIA,ARICA,"ESCUELA REPUBLICA DE FRANCIA , ARICA",-70.30892,-18.47422
9,11,ESC. GRAL. PEDRO LAGOS MARCHANT,ARICA,"ESC. GRAL. PEDRO LAGOS MARCHANT , ARICA",-70.31159,-18.49111
